<a href="https://www.kaggle.com/code/candacevogel/celeba-dataset-candace-vogel?scriptVersionId=262906571" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# CelebA Glasses Facial Characteristic Recognition

In [1]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms.v2 as transforms
from torchvision.transforms import ToTensor
from torchvision import datasets

from os import path
from PIL import Image

# Load data - create path to files

In [2]:
dataset_root = '/kaggle/input/celeba-dataset'

# input data from dataset

image_path = path.join(dataset_root, 'img_align_celeba/img_align_celeba')
# image_path contains the jpg files
attributes_path = path.join(dataset_root, 'list_attr_celeba.csv')
# file path for the csv file containing the attributes

# Read attributes CSV

In [3]:
df_attributes = pd.read_csv(attributes_path)
df_attributes.head()
# dataframe (df) is a type of table that panda library uses
# read csv file using pandas library ^
df_attributes.replace(-1, 0, inplace=True)
df_attributes['Eyeglasses'].head()

# this output will print just the eyeglasses data

0    0
1    0
2    0
3    0
4    0
Name: Eyeglasses, dtype: int64

In [4]:
# Replace -1 with 0

In [5]:

df_attributes.replace(-1, 0, inplace=True)
# 1 is true / represents wearing glasses
# 0 is false / represents not wearing glasses
# replaces every instance of -1 with 0
# now the data is all 0s and 1s
df_attributes['Eyeglasses'].head()
# prints just eyeglasses attributes column

0    0
1    0
2    0
3    0
4    0
Name: Eyeglasses, dtype: int64

# How many people are wearing glasses?

In [6]:
df_attributes['Eyeglasses'].value_counts()
# prints number of people wearing glasses in dataset

Eyeglasses
0    189406
1     13193
Name: count, dtype: int64

In [7]:
# Balance dataset

In [8]:
glasses_df = df_attributes [ df_attributes ['Eyeglasses'] == 1]
no_glasses_df = df_attributes [ df_attributes['Eyeglasses'] == 0]
# filtering attributes for glasses and no glasses

len(glasses_df), len(no_glasses_df)
# prints how many people are wearing glasses vs no glasses
glasses_count = len(glasses_df)
# prints how many people have glasses 
no_glasses_same_size = no_glasses_df.sample(glasses_count)
# sample is pandas method (pick rows at random)
len(no_glasses_same_size)
# prints how many people don't wear glasses in dataset
df_glasses_training = pd.concat ( [glasses_df, no_glasses_same_size], axis=0 )
# concatenate variables onto row axis
# axis=0 stacks tables vertically
# df_glasses_training is our balanced dataset
# the data is normalized so there is an equal number of images for glasses vs no glasses
# len(df_glasses_training)

df_glasses = df_glasses_training['Eyeglasses'].sample(10)
#display 10 rows at random
df_glasses.sample(10)
#only prints attributes we care about - eyeglasses data
# image_id and eyeglasses column



132155    1
11556     0
58575     1
141719    1
170363    0
177964    0
31871     1
110869    1
147678    0
19159     0
Name: Eyeglasses, dtype: int64